# IMPORTS

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import dayofweek, col

from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder

import numpy as np
import matplotlib.pyplot as plt

# LOAD DATA

You can download the data at [Kaggle site](https://www.kaggle.com/datasets/patrickzel/flight-delay-and-cancellation-dataset-2019-2023?select=2021.csv) or through the provided S3 bucket location.

In [2]:
# Initialize Spark session
spark = SparkSession.builder.appName("FlightDelayPrediction").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

your 131072x1 screen size is bogus. expect trouble
23/12/02 14:48:29 WARN Utils: Your hostname, JeevsPC resolves to a loopback address: 127.0.1.1; using 172.18.163.150 instead (on interface eth0)
23/12/02 14:48:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/02 14:48:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
file_path = "2021.csv"
data_2021 = spark.read.csv(file_path, header=True, inferSchema=True)

In [4]:
target_col = "DEP_DELAY"

# DATA ANALYSIS

In [5]:
data_2021.printSchema()

root
 |-- FL_DATE: date (nullable = true)
 |-- AIRLINE_CODE: string (nullable = true)
 |-- DOT_CODE: integer (nullable = true)
 |-- FL_NUMBER: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_CITY: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_CITY: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = tru

In [6]:
data_2021.show(5, truncate=False)

+----------+------------+--------+---------+------+----------------------+----+----------------------+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+------------+--------+--------+-----------------+-----------------+-------------+------------------+-----------------------+-------+--------+------+
|FL_DATE   |AIRLINE_CODE|DOT_CODE|FL_NUMBER|ORIGIN|ORIGIN_CITY           |DEST|DEST_CITY             |CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|DELAY_DUE_CARRIER|DELAY_DUE_WEATHER|DELAY_DUE_NAS|DELAY_DUE_SECURITY|DELAY_DUE_LATE_AIRCRAFT|FL_YEAR|FL_MONTH|FL_DAY|
+----------+------------+--------+---------+------+----------------------+----+----------------------+------------+--------+---------+--------+----------+---------+-------+------------+-

In [7]:
data_2021.describe().show()

+-------+------------+------------------+------------------+-------+------------+-------+------------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+--------------------+-----------------+--------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+-------------------+-----------------------+--------------------+-----------------+------------------+
|summary|AIRLINE_CODE|          DOT_CODE|         FL_NUMBER| ORIGIN| ORIGIN_CITY|   DEST|   DEST_CITY|      CRS_DEP_TIME|          DEP_TIME|        DEP_DELAY|          TAXI_OUT|        WHEELS_OFF|         WHEELS_ON|          TAXI_IN|     CRS_ARR_TIME|          ARR_TIME|        ARR_DELAY|           CANCELLED|CANCELLATION_CODE|            DIVERTED|  CRS_ELAPSED_TIME|      ELAPSED_TIME|          AIR_TIME|         DIST

In [8]:
for df_col in data_2021.columns:
    missing_count = data_2021.filter(data_2021[df_col].isNull()).count()
    print(f"{df_col}: {missing_count} missing values")

FL_DATE: 0 missing values


AIRLINE_CODE: 0 missing values


DOT_CODE: 0 missing values


FL_NUMBER: 0 missing values


ORIGIN: 0 missing values


...

DELAY_DUE_SECURITY: 4985066 missing values


DELAY_DUE_LATE_AIRCRAFT: 4985066 missing values


FL_YEAR: 0 missing values


FL_MONTH: 0 missing values


FL_DAY: 0 missing values


# DATA PREPROCESSING / FEATURE SELECTION

In [9]:
# Sort data by FL_DATE column in ascending order
preprocessed_data_2021 = data_2021.sort("FL_DATE")

# Remove rows that have cancelled flights
preprocessed_data_2021 = preprocessed_data_2021.filter(data_2021.CANCELLED == 0)

# Add column for day of the week, based on FL_DATE
preprocessed_data_2021 = preprocessed_data_2021.withColumn("DAY_OF_WEEK", dayofweek(col("FL_DATE")))

In [10]:
# Keep these columns in a new dataframe (Domain Knowledge Feature Selection)
preprocessed_data_2021 = preprocessed_data_2021.select("FL_YEAR", "FL_MONTH", "FL_DAY", "DAY_OF_WEEK", "AIRLINE_CODE", "ORIGIN", "DEST", "CRS_DEP_TIME", target_col)

preprocessed_data_2021 = preprocessed_data_2021.dropna()

In [11]:
preprocessed_data_2021.count()

5892337

In [12]:
# Convert categorical columns to OneHotEncoded vector
indexer = StringIndexer(inputCols=['AIRLINE_CODE', 'ORIGIN', 'DEST'], outputCols=['INDEXED_AIRLINE_CODE', 'INDEXED_ORIGIN', 'INDEXED_DEST'])
preprocessed_data_2021 = indexer.fit(preprocessed_data_2021).transform(preprocessed_data_2021)

encoder = OneHotEncoder(inputCols=['INDEXED_AIRLINE_CODE', 'INDEXED_ORIGIN', 'INDEXED_DEST'], outputCols=['ONEHOT_AIRLINE_CODE', 'ONEHOT_ORIGIN', 'ONEHOT_DEST'])
preprocessed_data_2021 = encoder.fit(preprocessed_data_2021).transform(preprocessed_data_2021)

In [13]:
# Keep these onehot encoded columns only
preprocessed_data_2021 = preprocessed_data_2021.select("FL_YEAR", "FL_MONTH", "FL_DAY", "DAY_OF_WEEK", 'ONEHOT_AIRLINE_CODE', 'ONEHOT_ORIGIN', 'ONEHOT_DEST', target_col)

In [14]:
# Take only first 10,000 rows, due to limited CPU power. If you have a better CPU or running this on AWS, you can comment this line out.
preprocessed_data_2021 = preprocessed_data_2021.limit(10000)

In [15]:
preprocessed_data_2021.show(5, truncate=False)

+-------+--------+------+-----------+-------------------+-----------------+-----------------+---------+
|FL_YEAR|FL_MONTH|FL_DAY|DAY_OF_WEEK|ONEHOT_AIRLINE_CODE|ONEHOT_ORIGIN    |ONEHOT_DEST      |DEP_DELAY|
+-------+--------+------+-----------+-------------------+-----------------+-----------------+---------+
|2021   |1       |1     |6          |(16,[6],[1.0])     |(370,[10],[1.0]) |(370,[76],[1.0]) |0.0      |
|2021   |1       |1     |6          |(16,[6],[1.0])     |(370,[10],[1.0]) |(370,[97],[1.0]) |-2.0     |
|2021   |1       |1     |6          |(16,[6],[1.0])     |(370,[139],[1.0])|(370,[10],[1.0]) |-4.0     |
|2021   |1       |1     |6          |(16,[6],[1.0])     |(370,[97],[1.0]) |(370,[21],[1.0]) |-5.0     |
|2021   |1       |1     |6          |(16,[6],[1.0])     |(370,[10],[1.0]) |(370,[148],[1.0])|14.0     |
+-------+--------+------+-----------+-------------------+-----------------+-----------------+---------+
only showing top 5 rows



In [16]:
preprocessed_data_2021.describe().show()

+-------+-------+--------+------+-----------+------------------+
|summary|FL_YEAR|FL_MONTH|FL_DAY|DAY_OF_WEEK|         DEP_DELAY|
+-------+-------+--------+------+-----------+------------------+
|  count|  10000|   10000| 10000|      10000|             10000|
|   mean| 2021.0|     1.0|   1.0|        6.0|            5.6873|
| stddev|    0.0|     0.0|   0.0|        0.0|43.635083599806194|
|    min|   2021|       1|     1|          6|             -39.0|
|    max|   2021|       1|     1|          6|            1779.0|
+-------+-------+--------+------+-----------+------------------+



In [17]:
# Vectorization
feature_cols = list(set(preprocessed_data_2021.columns) - set([target_col])) # All columns except ARR_DELAY (target variable)
vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
vectorized_data_2021 = vector_assembler.transform(preprocessed_data_2021)

In [18]:
vectorized_data_2021.select("features", target_col).show(5, truncate=False)

+--------------------------------------------------------------+---------+
|features                                                      |DEP_DELAY|
+--------------------------------------------------------------+---------+
|(760,[0,1,2,79,373,384,750],[2021.0,6.0,1.0,1.0,1.0,1.0,1.0]) |0.0      |
|(760,[0,1,2,49,373,393,755],[2021.0,6.0,1.0,1.0,1.0,1.0,1.0]) |-5.0     |
|(760,[0,1,2,100,373,384,750],[2021.0,6.0,1.0,1.0,1.0,1.0,1.0])|-2.0     |
|(760,[0,1,2,8,373,382,755],[2021.0,6.0,1.0,1.0,1.0,1.0,1.0])  |3.0      |
|(760,[0,1,2,13,373,513,750],[2021.0,6.0,1.0,1.0,1.0,1.0,1.0]) |-4.0     |
+--------------------------------------------------------------+---------+
only showing top 5 rows



In [19]:
# Standardization
scaler = StandardScaler(withMean=True, withStd=True, inputCol="features", outputCol="scaled_features")
scaled_data_2021 = scaler.fit(vectorized_data_2021).transform(vectorized_data_2021)

In [20]:
scaled_data_2021.select("scaled_features", target_col).show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# TRAIN/TEST SPLIT

In [21]:
# Define train/test split percentage
train_percent = 0.6
val_percent = 0.2
test_percent = 0.2

# Calculate the split indices
count = scaled_data_2021.count()
train_index = int(train_percent * count)
val_index = train_index + int(val_percent * count)

# Split the data into training, validation, and testing sets
train_data = scaled_data_2021.limit(train_index)
val_data = scaled_data_2021.limit(val_index).subtract(train_data)
test_data = scaled_data_2021.subtract(train_data).subtract(val_data)

# Extract train, validation, test sets from the dataset
X_train, y_train = train_data.select('scaled_features'), train_data.select(target_col)
X_val, y_val = val_data.select('scaled_features'), val_data.select(target_col)
X_test, y_test = test_data.select('scaled_features'), test_data.select(target_col)

# MODEL BUILDING

In [22]:
class RNN:
    def __init__(self, input_size, hidden_size, output_size, learning_rate):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.lr = learning_rate
        self.clip_value = 0.01
        
        # Initialize weights
        self.input_hidden_weights = np.random.randn(input_size, hidden_size)
        self.hidden_output_weights = np.random.randn(hidden_size, output_size)
        starting_bias = 0.1
        self.hidden_bias = np.array([starting_bias] * hidden_size)
        self.output_bias = np.array([starting_bias] * output_size)
    
    def leaky_relu(self, x):
        return np.where(x > 0, x, 0.01 * x)
    
    def clip_gradients(self, gradients):
        return np.clip(gradients, -self.clip_value, self.clip_value)

    def forward(self, x):
        self.hidden_state = self.leaky_relu(x.dot(self.input_hidden_weights) + self.hidden_bias) 
        output = self.hidden_state.dot(self.hidden_output_weights) + self.output_bias 
        return output

    def backward(self, x, output, target):
        loss_gradient = 2 * (output - target)  # Mean Squared Error loss gradient
        hidden_state_gradient = self.hidden_output_weights.dot(loss_gradient)
        
        # Clip gradients before updating weights
        hidden_state_gradient_clipped = self.clip_gradients(hidden_state_gradient)
        loss_gradient_clipped = self.clip_gradients(loss_gradient)
    
        self.input_hidden_weights -= self.lr * np.outer(x, hidden_state_gradient_clipped)
        self.hidden_output_weights -= self.lr * np.outer(self.hidden_state, loss_gradient_clipped)
        self.hidden_bias -= self.lr * hidden_state_gradient_clipped
        self.output_bias -= self.lr * loss_gradient_clipped
        
    def predict(self, x):
        self.hidden_state = self.leaky_relu(x.dot(self.input_hidden_weights) + self.hidden_bias)
        output = self.hidden_state.dot(self.hidden_output_weights) + self.output_bias
        return output

In [23]:
def mean_squared_error(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

# TRAINING

In [24]:
X_train_np, y_train_np = X_train.collect(), y_train.collect()

In [25]:
X_val_np, y_val_np = X_val.collect(), y_val.collect()

In [26]:
# Parameters
learning_rate = 0.001
epochs = 10
hidden_size = 1

input_size = len(X_train_np[0][0])
output_size = 1
model = RNN(input_size, hidden_size, output_size, learning_rate)

train_losses = []
val_losses = []

for epoch in range(epochs):
    print(f'Epoch {epoch + 1}:')
    
    # Training evaluation
    train_loss = 0
    for i in range(len(X_train_np)):
        input_train = np.array(X_train_np[i][0])
        target_train = y_train_np[i][0]
        output_train = model.forward(input_train)
        model.backward(input_train, output_train, target_train)
        train_loss += mean_squared_error(target_train, output_train)
    avg_train_loss = train_loss / len(X_train_np)
    train_losses.append(avg_train_loss)
    print(f'Train Loss: {avg_train_loss}', end=' | ')
    
    # Validation evaluation
    val_loss = 0
    for i in range(len(X_val_np)):
        input_val = np.array(X_val_np[i][0])
        target_val = y_val_np[i][0]
        output_val = model.forward(input_val)
        val_loss += mean_squared_error(target_val, output_val)
    avg_val_loss = val_loss / len(X_val_np)
    val_losses.append(avg_val_loss)
    print(f'Validation Loss: {avg_val_loss}\n')

Epoch 1:
Train Loss: 2014.8350517254528 | Validation Loss: 1513.6700124673596

Epoch 2:
Train Loss: 2019.7223624946394 | Validation Loss: 1515.5566644060186

Epoch 3:
Train Loss: 2020.2264434257215 | Validation Loss: 1515.9571511239242

Epoch 4:
Train Loss: 2020.4721005654717 | Validation Loss: 1516.2950387910068

Epoch 5:
Train Loss: 2020.6971382777085 | Validation Loss: 1516.4943840073965

Epoch 6:
Train Loss: 2020.9754008450316 | Validation Loss: 1516.802156422148

Epoch 7:
Train Loss: 2021.2189986754727 | Validation Loss: 1517.077505498349

Epoch 8:
Train Loss: 2021.46844118585 | Validation Loss: 1517.419963794543

Epoch 9:
Train Loss: 2021.6861310796883 | Validation Loss: 1517.7200232893001

Epoch 10:
Train Loss: 2021.9199878708203 | Validation Loss: 1518.041383181472



In [27]:
def plot_loss(train_loss, val_loss, hidden_size):
    # Create an array for the number of epochs, e.g., [1, 2, 3, ...]
    epochs = np.arange(1, len(train_loss) + 1)

    # Plot training accuracy and validation accuracy on the same graph
    plt.plot(epochs, train_loss, label='Training Loss', marker='o', linestyle='-')
    plt.plot(epochs, val_loss, label='Validation Loss', marker='o', linestyle='-')

    # Set axis labels and a legend
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='best')

    # Display the plot
    plt.xticks(np.arange(1, len(train_loss) + 1, step=1))
    plt.title(f'h = {hidden_size} | Training and Validation Loss (by Epoch)')
    plt.show()
    
plot_loss(train_losses, val_losses, hidden_size)

# EVALUATION

In [28]:
X_test_np, y_test_np = X_test.collect(), y_test.collect()

In [29]:
# Make predictions on a new set of data
preds = []

for i in range(len(X_test_np)):
    input = np.array(X_test_np[i][0])
    output = model.predict(input)
    preds.append(output)

# Convert predictions to a NumPy array
preds = np.array(preds)

In [30]:
mse_value = mean_squared_error(y_test_np, preds)
print(f"Test Mean Squared Error: {mse_value}")
print(f"Test Root Mean Squared Error: {np.sqrt(mse_value)}")

Test Mean Squared Error: 2418.9679025729815
Test Root Mean Squared Error: 49.18300420443003
